# Face Recognition Assignment
## Data Preprocessing

<span style="font-size: 20px;">For a neater, properly organized and easy-maintained assignment, this section covers all non-CNN related code, including:</span>

- **Importing Libraries**: Code for importing required Python libraries (e.g., pandas, numpy).
- **Dataset Loading**: Code for loading the dataset from a file or source.
- **Dataset Separation**: Code for splitting the dataset into training, validation, and test sets.

---

### Imoprt Libraries

In [134]:
import os
from glob import glob
import kagglehub
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.model_selection import train_test_split

SEED = 42
np.random.seed(SEED)

---

### Dataset Loading

In [138]:
def load_dataset():

    path = kagglehub.dataset_download("dmitrybabko/speech-emotion-recognition-en")+"/Crema"
    if not os.path.isdir(path):
        raise FileNotFoundError("Download CREMA dataset.")
    
    X_time = []
    X_freq = []
    y = []
    audio_files = sorted(glob(f"{path}/*.wav"))
    print(f'Audio Files Number: {len(audio_files)}')
    
    for audio_file in audio_files:
        audio, sr = librosa.load(audio_file)
        audio_fixed = librosa.util.fix_length(audio, size=int(5 * sr))
        
        zcr = librosa.feature.zero_crossing_rate(audio_fixed, frame_length=2048, hop_length=512)
        energy = librosa.feature.rms(y=audio_fixed, frame_length=2048, hop_length=512)
        mel_spec = librosa.feature.melspectrogram(y=audio_fixed, sr=sr)
        # mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

        X_time.append(np.vstack((zcr, energy)))
        X_freq.append(mel_spec)
        y.append(audio_file.split("_")[2])
        
    return X_time, X_freq, y

---

### Dataset Separation

In [160]:
def separate_dataset():
    # Time Domain Split
    X_time, X_freq, y = load_dataset()
    X_time_train_val, X_time_test, y_time_train_val, y_time_test = train_test_split(X_time, y, train_size=0.7, stratify=y, random_state=SEED)
    X_time_train, X_time_val, y_time_train, y_time_val = train_test_split(X_time_train_val, y_time_train_val, train_size=0.95, stratify=y_time_train_val, random_state=SEED)
    # Frequency Domain Split
    X_freq_train_val, X_freq_test, y_freq_train_val, y_freq_test = train_test_split(X_freq, y, train_size=0.7, stratify=y, random_state=SEED)
    X_freq_train, X_freq_val, y_freq_train, y_freq_val = train_test_split(X_freq_train_val, y_freq_train_val, train_size=0.95, stratify=y_freq_train_val, random_state=SEED)
    
    # Convert to NumPy arrays
    X_time_train = np.array(X_time_train)
    y_time_train = np.array(y_time_train)
    X_freq_train = np.array(X_freq_train)
    y_freq_train = np.array(y_freq_train)

    X_time_val = np.array(X_time_val)
    y_time_val = np.array(y_time_val)
    X_freq_val = np.array(X_freq_val)
    y_freq_val = np.array(y_freq_val)

    X_time_test = np.array(X_time_test)
    y_time_test = np.array(y_time_test)
    X_freq_test = np.array(X_freq_test)
    y_freq_test = np.array(y_freq_test)

    print(f'Training Set: time domain shape {X_time_train.shape}, frequency domain shape {X_freq_train.shape}')
    print(f'Validation Set: time domain shape {X_time_val.shape}, frequency domain shape {X_freq_val.shape}')
    print(f'Test Set: time domain shape {X_time_test.shape}, frequency domain shape {X_freq_test.shape}')
    
    return (X_time_train, X_freq_train, X_time_val, X_freq_val, X_time_test, X_freq_test, 
    y_time_train, y_freq_train, y_time_val, y_freq_val, y_time_test, y_freq_test)

In [166]:
(X_time_train, X_freq_train, X_time_val, X_freq_val, X_time_test, X_freq_test, 
 y_time_train, y_freq_train, y_time_val, y_freq_val, y_time_test, y_freq_test) = separate_dataset()


Audio Files Number: 7442
Training Set: time domain shape (4948, 2, 216), frequency domain shape (4948, 128, 216)
Validation Set: time domain shape (261, 2, 216), frequency domain shape (261, 128, 216)
Test Set: time domain shape (2233, 2, 216), frequency domain shape (2233, 128, 216)
